### Anvio - Contig profiling

Contig profiling creates a database of your contigs. It calculates k-mer frequencies for your sample (standard k-setting is 4, which you can change with the --kmer-size parameter (DON'T unless you have a good reason)), soft splits long contigs, and identifies open reading frames (which can be skipped using --skip-gene-calling). Run the following code to generate your database:

Subsequently, you can add various elements of analysis to your contig profile. The following list is available:

Augustus + Prodigal gene calls: adds open reading frames to your dataset from the genes from the Augustus database (eukaryotes) and Prodigal (bacteria + archaea) WORKING ON THIS: NOT SURE IF IT WORKS

Hidden Markov Model (HMM): A widely used prediction model in bioinformatics software, which can offer great advantages in homology detection.

NCBI's Cluster of Orthologous genes (NCBI COG): this allows you to annotate your database with gene functions from NCBI COG. Current version: 2020

KoFAM Metabolism calls: Uses the KEGG database to call metabolic genes and estimate paths of your community. Currently used KEGG version is KEGG_build_2020-12-23.

Kaiju Taxonomy calls:

Each of these adds a new layer of information to your dataset, so might be very interesting to explore.

In [ ]:
conda activate anvio-7.1

#install taxonomy 
diamond --version
conda install diamond=0.9.14
anvi-setup-scg-taxonomy

#download ncbi-cogs 
anvi-setup-ncbi-cogs --num-threads 11

#metabolism database
anvi-setup-kegg-kofams
#not working - says command not found 

In [ ]:
#!/bin/bash
#SBATCH -c 24  # Number of Cores per Task
#SBATCH --mem=50G  # Requested Memory
#SBATCH -p cpu  # Partition
#SBATCH -t 20:00:00  # Job time limit
#SBATCH --mail-type=ALL
#SBATCH -o //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/bash_scripts/slurm-%j.out  # %j = job ID

module load miniconda/22.11.1-1
conda activate anvio-8

#Contig database from assembled genomes. stores information related to your sequences: positions of open reading frames, k-mer frequencies for each contig, functional and taxonomic annotation of genes, etc.
#set parameters:
SAMPLENAME=mcav
CONTIGPATH=/project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/results/mcav_assembly3
CONTIGFILE=mcav.contigs-fixed.fa

#generate the contigs database:
#default k-mer frequency is 4
anvi-gen-contigs-database -f $CONTIGPATH/$CONTIGFILE --project-name $SAMPLENAME -o ./working/$SAMPLENAME.contigs.db  

#integrate HMMs into the database:
anvi-run-hmms -c ./working/$SAMPLENAME.contigs.db --num-threads 6

#this runs NCBI COGs against your contigs.db, integrating gene functions.
anvi-run-ncbi-cogs -c ./working/$SAMPLENAME.contigs.db -T 4

#ADD KEGG-KOFAM
anvi-run-kegg-kofams -c ./working/$SAMPLENAME.contigs.db \
                     -T 4 #these are the threads that Anvi'O is allowed to use
#ADD CONTIG STATS
anvi-display-contigs-stats ./working/$SAMPLENAME.contigs.db --report-as-text --as-markdown -o ./results/anvio_stats

#generates contig database from merged, fixed contig fasta file created in previous step..need for downstream analysis
#JOB ID: 13297970
#bash script: mcav_db.txt

In [ ]:
#below: create sample profiles
module load miniconda/22.11.1-1
conda activate anvio-8

#set parameters
SAMPLENAME=mcav
#mkdir=//project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/$SAMPLENAME_profiles
samplepath=//project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/bash_scripts
OUTDIR=//project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/index

for f in T1_12_2022 T1_13_2022 T1_16_2019 T1_20_2019 T1_24_2019 T1_40_2022 T1_57_2022 T1_70_2022 T2_10_2022 T2_16_2019 T3_13_2022 T3_14_2019 T3_15_2019 T3_19_2022 T3_1_2019 T3_40_2022 T3_48_2022 T3_49_2022 T3_51_2022 T3_60_2022 T3_8_2019 T3_9_2019
do
anvi-profile -c ./working/$SAMPLENAME.contigs.db  \
            -i ./working/index/"$f".bam \
            --min-percent-identity 95 \
            --sample-name "$f"
            --output-dir $OUTDIR
#use contig database and sample bam files to create single profiles. can specify to keep contigs of min length (min-contig-length) and 95% identity to database
#keep parameters consistent in order to merge to larger profile 
#already filtered out min 1000bp contigs in 'anvi-script-reformat-fasta', so not adding specification.
done

#seperated individual sample profiling and merging to different bash scripts just for troubleshooting
#bash script: profiles.txt
#JOB ID: 13298522

In [ ]:
SAMPLENAME=mcav
OUTDIR=//project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/index
anvi-display-contigs-stats working/$SAMPLENAME.contigs.db --report-as-text -o results/anvi_display_stats.txt

contigs_db	mcav_contigs_fixed
Total Length	57097694
Num Contigs	39414
Num Contigs > 100 kb	0
Num Contigs > 50 kb	0
Num Contigs > 20 kb	2
Num Contigs > 10 kb	35
Num Contigs > 5 kb	370
Num Contigs > 2.5 kb	2156
Longest Contig	24857
Shortest Contig	1000
Num Genes (prodigal)	58029
L50	14211
L75	25911
L90	33821
N50	1339
N75	1127
N90	1044
Archaea_76	9
Bacteria_71	8
Protista_83	5
Ribosomal_RNA_12S	0
Ribosomal_RNA_16S	0
Ribosomal_RNA_18S	2
Ribosomal_RNA_23S	0
Ribosomal_RNA_28S	2
Ribosomal_RNA_5S	0
bacteria (Bacteria_71)	0
eukarya (Protista_83)	0
archaea (Archaea_76)	0

**Do not use below code...still troubleshooting**

In [ ]:
#!/bin/bash
#SBATCH -c 24  # Number of Cores per Task
#SBATCH --mem=50G  # Requested Memory
#SBATCH -p cpu  # Partition
#SBATCH -t 6:00:00  # Job time limit
#SBATCH --mail-type=ALL
#SBATCH -o //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/bash_scripts/slurm-%j.out  # %j = job ID

module load miniconda/22.11.1-1
conda activate anvio-8
SAMPLENAME=mcav
OUTDIR=//project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/working/index
DIR=//project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/results/MetaBAT_mcav_bins


anvi-merge -c ./working/"$SAMPLENAME".contigs.db \
            $OUTDIR/*/PROFILE.db \
            -o $OUTDIR/"$SAMPLENAME"_profile_merged
#merge single sample profiles to one profile

FILES=$(find $DIR/*.fa)
for f in $FILES; do
 NAME=$(basename $f .fa)
 grep ">" $f | sed 's/>//' | sed -e "s/$/\t$NAME/" | sed 's/\./_/' >> metabins4anvio.txt
done
#convert metabin results to anvio format
#metabin produces fasta files containing contigs of each bin 
#collection artifact requires a txt file that contains list of contigs with their associated bins (2 columns) 

anvi-import-collection metabins4anvio.txt \
                       -p $OUTDIR/"$SAMPLENAME"_profile_merged/PROFILE.db \
                       -c ./working/$SAMPLENAME.contigs.db \
                       --contigs-mode \
                       -C "$SAMPLENAME"_collection
#import binning results of metabat from 3Binning step as a collection artifact in anvio
##contigs-mode specificies that input txt file describes contigs names not split names
  

#seperated individual sample profiling and merging to different bash scripts just for troubleshooting
#bash script: binning
#job ID: 

**Troubleshooting code below**
- Trying to figure out clustering in anvio itself
- Skip for now

In [ ]:
anvi-cluster-contigs -p $OUTDIR/"$SAMPLENAME"_contigs_merged \
                    -c ./working/$SAMPLENAME.contigs.db \
                    --log-file //project/pi_sarah_gignouxwolfsohn_uml_edu/brooke/bash_scripts/cluster_log \
                    -C "$SAMPLENAME"_collection \
                    -T 3 \
                    --driver 'metabat2'
#binning: clusters contigs. can use various drivers: 'concoct, metabat2, maxbin2, dastool, or binsanity.'

#error output that the module of clustering isnt fully developed and is better to just use own binning software. use flag '--just-do-it' if you want to try it out

#### Results from importing collection:
- low coverage...make metabin parameters less stringent??

In [ ]:
anvi-migrate --migrate-safely ./working/"$SAMPLENAME".contigs.db
#migrate anvio artifact after updating anvio to version 8 - only do once if updated

#### Taxonomy 
https://merenlab.org/2019/10/08/anvio-scg-taxonomy/
- single-copy core genes (SCGs) and the taxonomy of the genomes - as defined by the GTDB - from which these genes are coming from

- very bad results...don't think we should use this method?

In [ ]:
mkdir ./results/taxonomy
SAMPLENAME=mcav 

In [ ]:
anvi-estimate-scg-taxonomy -c ./working/$SAMPLENAME.contigs.db -p working/index/"$SAMPLENAME"_profile_merged/PROFILE.db -T 3 --metagenome-mode --compute-scg-coverages -o ./results/taxonomy/"$SAMPLENAME"-scg-taxonomy.tsv
#taxa matching from assembled metagenome - will just match ASVs to SCG taxa and give percent identity 

In [ ]:
anvi-run-scg-taxonomy -c working/"$SAMPLENAME".contigs.db

In [ ]:
anvi-run-scg-taxonomy -c working/"$SAMPLENAME".contigs.db --min-percent-identity 60

In [ ]:
anvi-estimate-scg-taxonomy -c ./working/"$SAMPLENAME".contigs.db \
                           -p ./working/index/mcav_profile_merged/PROFILE.db \
                           --num-threads 3 \
                           --metagenome-mode \
                           --compute-scg-coverages \
                           --output-file ./results/taxonomy/"$SAMPLENAME"-taxa-abundance.tsv
#difference here is that it will calculate relative abundances of the matched SCG taxa across all samples as well as percent identity (like an OTU table)

In [ ]:
ssh -L 8080:localhost:8080 unity
#coonect to unity this way from local terminal if you want to use the interactive browser below

In [ ]:
anvi-interactive -p $OUTDIR/"$SAMPLENAME"_profile_merged/PROFILE.db -c ./working/$SAMPLENAME.contigs.db -C "$SAMPLENAME"_collection
#need collection info since we skipped clustering in the merging step 
#also very bad results - don't think it's helpful